# **Construção de um programa de fidelidade com clusterização de clientes.**

In [ ]:
#Abrir Conexão
import sqlite3
import pandas as pd
conn = sqlite3.connect ("database.db")

# consulta dos dados no banco de dados
consulta_db = """
 SELECT
 *
 FROM
 flight_activity fa LEFT JOIN flight_loyalty_history flh ON(fa.loyalty_number = flh.loyalty_number)
"""
df_atividade = pd.read_sql_query( consulta_db, conn )

#Fecha conexão
conn.close()

# comando para mostrar a tabela resultante da consulta
df_atividade.head()

# **Preparação dos dados**

In [ ]:
# Numero de dados faltantes em cada coluna
df_atividade.isna().sum()

#Selecionando somente as colunas numéricas, porém necessita a última coluna, que é o rótulo que será passado para o algoritmo aprender.
#Essas colunas foram escolhidas por serem as melhores para o algoritmo de ML e possuem características das pessoas no banco de dados.
colunas=["year", "month", "flights_booked", "flights_with_companions", "total_flights", "distance", "points_accumulated", "salary",
 "clv", "loyalty_card"]

#Guardar a seleção (tabela com os dados que queremos) acima em uma variável para o treinamento do algoritmo
df_colunas_numericas = df_atividade.loc[:, colunas]

#Remover as linhas que contém dados faltantes, utilizando a variável acima e colocando em outra variável.
df_dados_completos = df_colunas_numericas.dropna()

# verificando o numero de linhas vazias
df_dados_completos.isna().sum()

In [ ]:
#Verificando quantas linhas sobraram para realizarmos o algoritmo de ML
df_dados_completos.shape[0]

In [ ]:
df_dados_completos.head()

# ***Machine Learning***

In [ ]:
from sklearn import tree as tr

X_atributos = df_dados_completos.drop(columns = "loyalty_card") #Tiramos essa coluna pois ela contém strings
y_rotulos = df_dados_completos.loc[:, "loyalty_card"] #Aqui usaremos o y como rótulo.

#Definição do algoritmo
modelo = tr.DecisionTreeClassifier(max_depth=5) #=2 é o nv da árvore de decisão. Vemos isso executando o cód tr.plot abaixo.

#Treinamento do algoritmo
modelo_treinado = modelo.fit(X_atributos, y_rotulos)

In [ ]:
tr.plot_tree(modelo_treinado, filled = True); #É o plot do algoritmo treinado, acrescentando ';', os comentários do aprendizado são removidos, ficando somente o plot.

# **Apresentando os resultados**

In [ ]:
#Pegando aleatoriamente um cliente dentro do dataframe já tratado (df_dados_completos)
X_novo = X_atributos.sample()

#Trás a probabilidade do cliente poder aceitar um dos programas de fidelidade oferecidos, e essas probabilidades obedecem uma ordem alfabética.
#Como temos 3 programas, a porcentagem virá nessa ordem: Aurora, Nova e Star.
previsao = modelo_treinado.predict_proba(X_novo)

print("Probabilidade:\nAurora: {:.1f}% \nNova: {:.1f}% \nStar: {:.1f}%".format(100*previsao[0][0], 100*previsao[0][1], 100*previsao[0][2])) #[0][0] é do 1º elemento...

# **Painel de visualização**

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import numpy as np

In [ ]:
#Função recebe entradas dos usuários, chama o algoritmo, executa a previsão e devolve para o painel o resultado
def predict(*args):
  X_novo = np.array( [args] ).reshape(1, -1) #args = argumentos, ajeitando as colunas
  previsao = modelo_treinado.predict_proba(X_novo)

  return {"Aurora": previsao[0][0], "Nova": previsao[0][1], "Star": previsao[0][2]}


#Criação de um painel de visualização, de inserção de dados e resposta com a probabilidade
with gr.Blocks() as demo: #Cada bloco deve se iniciar com 'with'
  #Título do painel
  gr.Markdown( """ # PROPENSAO DE COMPRA """)

  with gr.Row():
    with gr.Column():
      gr.Markdown( """ # Atributos do Cliente """)
      year                      = gr.Slider( label="year", minimum=2017, maximum=2018, step=1, randomize=True) #Para descobrir o min e o max, pode usar o 'X_atributos.loc[:, 'year'].min() ou max()
      month                     = gr.Slider( label="month", minimum=1, maximum=12, step=1, randomize=True)
      flights_booked            = gr.Slider( label="flights_booked", minimum=0, maximum=21, step=1, randomize=True)
      flights_with_companions   = gr.Slider( label="flights_with_companions", minimum=0, maximum=11, step=1, randomize=True)
      total_flights             = gr.Slider( label="total_flights", minimum=0, maximum=32, step=1, randomize=True)
      distance                  = gr.Slider( label="distance", minimum=0, maximum=6293, step=1, randomize=True)
      points_accumulated        = gr.Slider( label="points_accumulated", minimum=0.00, maximum=676.50, step=1, randomize=True)
      salary                    = gr.Slider( label="salary", minimum=58486.00, maximum=407228.00, step=0.1, randomize=True)
      clv                       = gr.Slider( label="clv", minimum=2119.89, maximum=83325.38, step=0.1, randomize=True)

      with gr.Row():
        #gr.Markdown( """ # LINHA 1""")

        with gr.Row():
          gr.Markdown( """ # Botao de Previsao """)
          predict_btn = gr.Button(value = "Previsao")

    with gr.Column():
      gr.Markdown( """ # Propensao de Compra do Cliente """)
      label = gr.Label()

  #Botão predict, recebe os dados inseridos pelo usuário, aciona o algoritmo de ML treinado e exibe o resultado
  predict_btn.click(
      fn=predict,
      inputs=[
          year,
          month,
          flights_booked,
          flights_with_companions,
          total_flights,
          distance,
          points_accumulated,
          salary,
          clv
          ],
      outputs=[label] )

demo.launch(share=True)